In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import shap
from scipy.stats.mstats import winsorize
from scipy.stats import shapiro, mannwhitneyu
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (mean_squared_error, r2_score, 
    classification_report, confusion_matrix, accuracy_score,
    roc_curve, auc)

/Users/arshsingh/Documents/Montclair/portfolio-masters-capstone/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load the dataset
df = pd.read_csv('combined.00-25.csv')

In [5]:
# Fill missing/empty 'totalNonCurrentLiabilities' with 'totalLiabilities'
df['totalCurrentLiabilities'] = df['totalCurrentLiabilities'].fillna(df['totalLiabilities'])

# df.loc[df['totalEquity'] == 0, 'totalEquity'] = (
#     df['totalLiabilitiesAndTotalEquity'] - df['totalLiabilities']
# )

# Creating metrics and ratios.
df['totalEquity'] = df['totalLiabilitiesAndTotalEquity'] - df['totalLiabilities']
df['currentRatio'] = df['totalCurrentAssets'] / df['totalCurrentLiabilities']
df['debtToEquity'] = df['totalDebt'] / df['totalEquity']
df['ROE'] = df['netIncome_x'] / df['totalEquity']
df['inventoryTurnover'] = df['costOfRevenue'] / df['inventory_x']
df['Return'] = df.groupby('symbol_stock')['Adj Close'].pct_change()
df['ROA'] = df['netIncome_x'] / df['totalAssets']
df['free_cash_flow_yield'] = df['freeCashFlow'] / df['totalEquity']

df['Profit_Margin'] = df['netIncome_x'] / df['revenue']
df['Op_Cash_Flow_to_Revenue'] = df['operatingCashFlow'] / df['revenue']

if 'dividendsPaid' in df.columns and 'netIncome_x' in df.columns:
    df['retained_earnings'] = df['netIncome_x'] - df['dividendsPaid']
    # Compute a ratio of what fraction of net income is retained.
    df['re_ratio'] = df['retained_earnings'] / df['netIncome_x']

df['capital_light'] = df['capitalExpenditure'] / df['operatingCashFlow']

# df['re_ratio'] = df[ 'retainedEarnings'] / df['netIncome_x']

# Metrics after looking at Sector-level correlation
df['goodwillIntangible_to_assets'] = df['goodwillAndIntangibleAssets'] / df['totalAssets']
df['sellingMarketing_to_revenue'] = df['sellingAndMarketingExpenses'] / df['revenue']
df['totalInvestments_to_assets'] = df['totalInvestments'] / df['totalAssets']
df['rnd_to_revenue'] = df['researchAndDevelopmentExpenses'] / df['revenue']
df['inventory_to_assets'] = df['inventory_x'] / df['totalAssets']

In [6]:
# 2. Data Cleaning & Date Conversion
# Use 'acceptedDate_x' as the release date of the financial statement.
df['accepted_date'] = pd.to_datetime(df['acceptedDate_x'])


# Define Future Return Horizon - in days - for the target return.
# 365 days for a 1-year return. Change to 3*365 or 5*365
horizon_days = 365 * 5

# Create a new column that marks the target date from where future return will be calculated.
df['target_date'] = df['accepted_date'] + pd.Timedelta(days=horizon_days)

df = df.sort_values(by=['symbol_stock', 'accepted_date'])

# Attach Future Price Using Merge_asof by creating a separate dataframe containing each company's 
# accepted_date and Adj Close, then rename columns so it's possible to merge on time.
df_prices = df[['symbol_stock', 'accepted_date', 'Adj Close']].rename(
    columns={'accepted_date': 'future_date', 'Adj Close': 'future_price'}
)

df = df.sort_values('target_date')

df_prices = df[['symbol_stock', 'accepted_date', 'Adj Close']].rename(
    columns={'accepted_date': 'future_date', 'Adj Close': 'future_price'}
).sort_values('future_date')

print(df['accepted_date'].isnull().sum())
print(df['target_date'].isnull().sum())

df = df.dropna(subset=['accepted_date'])
df['target_date'] = df['accepted_date'] + pd.Timedelta(days=horizon_days)
df_prices = df_prices.dropna(subset=['future_date'])

df_model = pd.merge_asof(
    left=df,
    right=df_prices,
    left_on='target_date',
    right_on='future_date',
    by='symbol_stock',
    direction='forward'
)

# Compute Future Return by calculating the percentage return over the horizon.
df_model['Return_future'] = (df_model['future_price'] - df_model['Adj Close']) / df_model['Adj Close']

# Remove rows where the future price and future return is not available.
df_model = df_model.dropna(subset=['Return_future']).copy()

# Define the Target Variable by creating a binary target: 
# label as 1 if the future return is positive (good stock), and 0 otherwise.
# df_model['good_stock'] = (df_model['Return_future'] > 0.0825).astype(int)
df_model['good_stock'] = (df_model['Return_future'] > 0.25).astype(int)

17
17
